In [1]:
import psycopg2
import json
from io import StringIO
import os

import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# Get db config
with open('../../../../context/SMI/config/postgres.config') as config_file:
    db_config = json.load(config_file)

# Local database deployment
conn = psycopg2.connect(
                        dbname=db_config['db_name'],
                        user=db_config['db_user'],
                        host='localhost',
                        port=db_config['db_port'],
                        password=db_config['db_password'],
                        options=db_config['db_options']
                        )
conn.autocommit = True
cur = conn.cursor()

schema = db_config['db_schema']

In [3]:
# Create directories:
if not os.path.isdir(logs_path):
    print('Environment job: Creating ' + app_name + ' logs directory')
    os.makedirs(logs_path)

if not os.path.isdir(temp_data_path):
    print('Environment job: Creating ' + app_name + ' data directory')
    os.makedirs(temp_data_path)

In [4]:
# Directories check:
print('Environment job: Checking folder '+ logs_path)
if os.path.isdir(logs_path):
    print('Environment job: Folder '+ logs_path + ' exists')
else:
    print('Environment job: Folder '+ logs_path + ' does not exists')
    print('Environment job: Creating folder '+ logs_path)
    os.makedirs(logs_path)

print('Environment job: Checking folder '+ temp_data_path)
if os.path.isdir(temp_data_path):
    print('Environment job: Folder '+ temp_data_path + ' exists')
else:
    print('Environment job: Folder '+ temp_data_path + ' does not exists')
    print('Environment job: Creating folder '+ temp_data_path)
    os.makedirs(temp_data_path)

Environment job: Checking folder logs/
Environment job: Folder logs/ exists
Environment job: Checking folder ../data/
Environment job: Folder ../data/ exists


In [9]:
class DatabaseCreation:
    '''
    Database creation and initial data insertion:
    '''
    def __init__(self,
                queries_path,
                conn,
                schema
                ):
        self.queries_path = queries_path
        self.conn = conn
        self.cur = cur
        self.schema = schema

    @staticmethod
    def get_info(url, header):
        '''
        Function to get the initial twitter users file.
        Params: 
            - url: list of url pages with the most followed spanish twitter accounts.
            - header: specifies robots.txt user agent.
        Output: Uncleaned dataframe with parsed tables.
        '''
        try:

            #Get text from url:
            page = requests.get(url, headers=header)
            soup = BeautifulSoup(page.text, "lxml")
            results = soup.find(id="listado")

            #Get table from text:
            df = pd.DataFrame([[tr for tr in tab] for tab in results.table])
            df = df.loc[:,1:]

            #Get column names:
            colnames = [str(name) for name in df.iloc[0]]
            colnames = [re.search('<b>(.*)</b>', name).group(1).replace('<br/>', ' ') for name in colnames if name != 'None']
            df = df.loc[1:,2:]
            df.columns = colnames

            #Dropping columns:
            df = df[~df['Twittero'].isnull()].reset_index(drop=True)
            return(df)

        except:
            print('Get info from XML files error')
            return 1

    @staticmethod
    def get_user(input):
        '''
        Function to extract user name from the list.
        Params:
            - input: Twitter message.
        Output: Twitter user.
        '''
        try:
            #Locating the username:
            s = str(input)
            start = s.find(">@") + len(">@")
            end = s.find("<br/")
            substring = s[start:end]

            #Returning the username from:
            return(substring)

        except:
            print('Get username from XML format error')
            return 1

    @staticmethod
    def get_n(input):
        '''
        Function to extract numeric values from the table:
        Params:
            - input: Different values from twitter users accounts.
        Output: Value
        '''
        try:
            #Locating the value;
            s = str(input)
            start = s.find(">") + len(">")
            end = s.find("</td")
            substring = s[start:end]

            #Returning the value:
            return(substring.replace(',', ''))

        except:
            print('Get numeric variables from XML format error')
            return 1

    def get_initial_users_table(self, urls, headers):
        '''
        Function to create the initial users dataframe.
        Params:
            - url: list of url pages with the most followed spanish twitter accounts.
            - header: specifies robots.txt user agent.
        Output: Dataframe with cleaned data.
        '''
        try:
            #Scrap info from wp:
            info = self.get_info(urls, headers)

            #Columns to clean:
            cols = ['Twittero', 'Seguido por', 'Sigue a', 'Tweets', 'Twitea desde', 'Ultimo Tweet', 'Categoria']

            #Clean columns:
            for col in cols:
                if col == 'Twittero':
                    users = [self.get_user(info[col][i]) for i in range(info.shape[0])]
                    info[col] = users
                else:
                    info[col] = [self.get_n(info[col][i]) for i in range(info.shape[0])]
            return(info)

        except:
            print('Get initial users table')
            return 1

    def get_tw_users_list(self, urls, headers, ini_users_dict):
        '''
        Function to get all users from different urls
        Params:
            - url: list of url pages with the most followed spanish twitter accounts.
            - header: specifies robots.txt user agent.
        Output: Users list and users table
        '''
        #Get all users from tables of different sections:
        try:
            users = []
            df_out = pd.DataFrame()
            for i in range(len(urls)):
                
                #Create users dataframe and users list:
                df = self.get_initial_users_table(urls[i][0], headers)
                df_out = pd.concat([df_out, df], axis=0).reset_index(drop=True)
                users.extend(df['Twittero'].to_list())

            #Drop duplicates:
            if len(users) != len(set(users)):
                users = list(set(users))
                df_out = df_out.drop_duplicates().reset_index(drop=True)

            #Formating columns:
            df_out.columns = [key for key in list(ini_users_dict.keys())]
            df_out = df_out.astype(ini_users_dict)
            df_out['lastTweet'] = np.where(df_out['lastTweet']=='n/d', df_out['tweetsSince'], df_out['lastTweet'])
            df_out['tweetsSince']=pd.to_datetime(df_out['tweetsSince'])
            df_out['lastTweet']=pd.to_datetime(df_out['lastTweet'])
            df_out = df_out.fillna(0)
            return(df_out)

        except:
            print('Get twitter users list error')
            return 1

    @staticmethod
    def filter_usrs_loc(df, munlist):
        '''
        Function to filter the location field given a municipalities list, to ensure spanish users:
        params:
            - df: input dataframe with users information:
            - munlist: list of municipalities:
        Output: filtered users table.
        '''
        #Convert location field to lower case:
        df['location'] = df['location'].apply(lambda r: r.replace(',', ''))
        df['location'] = df['location'].apply(lambda r: r.lower())

        #Filter location:
        df = df[df['location'].isin(munlist)]
        
        return(df)

    def backup_check(self, path, db_munlist, kind):
        '''
        Function to check whether there are backups.
        params:
            - path: relative path to the backup file.
            - kind: initial users or users.
        Output: 
            - df: users dataframe.
            - usr_ls: list of screenName users.
            - check: boolean to check whether there are backup or not.
        '''
        print('Data job: Check if ' + kind + ' backup exists.')
        if os.path.isfile(path):

            print('Data job: ' + kind + ' backup exists. Loading file: ' + path)
            with open(path, 'r') as f:

                df = pd.json_normalize(json.load(f))
                if kind == 'users':
                    print('Data Engineering job: Filtering location from backup users.')
                    print('Data Engineering job: Observations before filter: ' + str(df.shape[0]))
                    df = self.filter_usrs_loc(df, db_munlist)
                    df['ff_lookup'] = False
                    print('Data Engineering job: Observations after filter: ' + str(df.shape[0]))
                usr_ls = df['screenName'].to_list()
                check = True
                print('Data job: ' + kind + ' backup from json file retrieved.')
        else:

            print('Data job: ' + kind + ' backup does not exists. ')
            df = pd.DataFrame()
            usr_ls = []
            check = False
        
        return(df, usr_ls, check)

    @staticmethod
    def fetchone_SQL(path):
        """
        Function to fetch one observation from a query to database:
        params:
            - path: relative path to the file.
        """
        with open(path, 'r') as f:
            query = f.read().format(schema=schema)
            try:
                cur.execute(query)
                return(cur.fetchone()[0])
            except (Exception, psycopg2.DatabaseError) as error:
                conn.rollback()
                cur.close()
                print(error) 
    
    @staticmethod
    def fetchall_SQL(path):
        """
        Function to fetch all observations from a query to databasee:
        params:
            - path: relative path to the file.
        """
        with open(path, 'r') as f: 
            query = f.read().format(schema=schema)
            try:
                cur.execute(query)
                db_fetch = cur.fetchall()
                db_fetch = [db_fetch[i][0] for i in range(len(db_fetch))]
                return(db_fetch)
            except (Exception, psycopg2.DatabaseError) as error:
                conn.rollback()
                cur.close()
                print(error)

    @staticmethod
    def query_SQL(path):
        """
        Function to make a query to database:
        params:
            - path: relative path to the file.
        """
        with open(path, 'r') as f:
            query = f.read().format(schema=schema)
            try:
                cur.execute(query)
            except (Exception, psycopg2.DatabaseError) as error:
                conn.rollback()
                cur.close()
                print(error)

    @staticmethod
    def df_to_postgres(conn, df, table):
        """
        Function to save dataframe into postgres with copy_from:
        params:
            - conn: database connection.
            - df: pandas dataframe.
            - table: database table.
        """
        try:
            #Buffering the dataframe into memory:
            buffer = StringIO()
            df.to_csv(buffer, header=False, index=False)
            buffer.seek(0)

            #Copy cached dataframe into postgres:
            cur = conn.cursor()
            cur.copy_from(buffer, table, sep=",")
            conn.commit()
            
        except (Exception, psycopg2.DatabaseError) as error:
            conn.rollback()
            cur.close()
            print(error)
            return 1
        cur.close()

    def db_cs(self):
        '''
        Function to check and create the database schema and tables.
        params: selft referenced, no params.
        '''
        # Check if the schema exists.
        print('Database job: Check if SMI schema exists.')
        schema_check = self.fetchone_SQL(queries_path + 'SMI_schema_check.sql')

        # If schema exist, check tables.
        if schema_check:

            # Check initial users table.
            print('Database job: SMI schema exists on DB.')
            print('Database job: Check if initial users table exist on DB.')
            db_ini_check = self.fetchone_SQL(queries_path + 'SMI_ini_users_check.sql')

            # If exists, do nothing.
            if db_ini_check:
                print('Database job: Initial users table exist on DB.')

            # If it does not exist, create initial users table.
            else:
                print('Database job: Initial users table does not exist on DB.')
                print('Database job: Creating initial users table on DB.')
                self.query_SQL(queries_path + 'SMI_ini_users_table_creation.sql')
                print('Database job: Initial users table created on DB.')

            # Check users table.
            print('Database job: Check if users table exist on DB.')
            db_usr_check = self.fetchone_SQL(queries_path + 'SMI_usrs_table_check.sql')

            #If exists, do nothing.
            if db_usr_check:
                print('Database job: Users table exist on DB.')

            #If it does not exist, create users table.
            else:
                print('Database job: Users table does not exist on DB.')
                print('Database job: Creating users table on DB.')
                self.query_SQL(queries_path + 'SMI_usrs_table_creation.sql')
                print('Database job: Users table created on DB.')

        # If schema does not exists, create schema and tables.
        else:
            print('Database job: SMI schema does not exist.')
            print('Database job: Cold start - Creating SMI schema and tables on DB.')
            self.query_SQL(queries_path + 'SMI_coldstart_database.sql')
            print('Database job: DB schema and tables created.')

    def scrap_users(self):
        '''
        Function to scrap initial users from url:
        params: self referenced, no params.
        '''
        # Scrap ini users, create backup and fill database table:
        print('Scraping job: Retrieve initial users from url.')
        df = self.get_tw_users_list(urls, headers, ini_users_dict)
        print('Scraping job: Initial users from url retrieved.')
        print('Scraping job: Save initial users to json backup.')
        df.to_json(temp_data_path + 'db_ini_users_' + db_today + '.json', orient='records', date_format='iso')
        self.df_to_postgres(conn, df, initial_users_table)
        print('Database job: Initial users table created on DB.')

    def insert_ini_users(self):
        '''
        Function to insert initial users into DB.
        params: self referenced, no params.
        '''
        db_ini_ls = self.fetchall_SQL(queries_path + 'SMI_ini_database_screenName.sql')
        path_ini = temp_data_path + 'db_ini_users_' + db_ini_users_bkp + '.json'
        df_ini_users, df_ini_ls, df_ini_user_check = self.backup_check(path_ini, db_munlist, kind = 'initial users')

        if df_ini_user_check:
            
            if (len(db_ini_ls) == 0):
                print('Database job: Initial users table is empty.')
                print('Database job: Insert initial users backup into DB.')
                self.df_to_postgres(conn, df_ini_users, initial_users_table)
            else:
                print('Database job: Initial users table is not empty.')
                print('Data job: Compare initial users on DB and backup.')

                df_ini_ls.sort()
                db_ini_ls.sort()
                
                if df_ini_ls == db_ini_ls:
                    print('Data job: initial users match.')
                else:
                    print('Database job: initial users do not match, drop and create initial users table.')
                    #Create initial users table:
                    print('Database job: Creating initial users table on DB.')
                    self.query_SQL(queries_path + 'SMI_ini_users_table_creation.sql')
                    
                    #Initial users table insertion:
                    print('Database job: Insert initial users back into DB.')
                    self.df_to_postgres(conn, df_ini_users, initial_users_table)
                    print('Database job: Initial users table inserted on DB.')
        else:
            self.scrap_users()

    def insert_users(self):
        '''
        Function to insert users backup into DB.
        params: self referenced, no params.
        '''
        db_usr_ls = self.fetchall_SQL(queries_path + 'SMI_usrs_database_screenName.sql')
        path_usr = temp_data_path + 'db_users_' + db_users_bkp + '.json'
        df_usr, df_usr_ls, df_usr_check = self.backup_check(path_usr, db_munlist, kind = 'users')

        if df_usr_check:
            
            if (len(db_usr_ls) == 0):
                print('Database job: Users table is empty.')
                print('Database job: Insert users backup into DB.')
                self.df_to_postgres(conn, df_usr, users_table)
            else:
                print('Database job: Users table is not empty.')
                print('Data job: Compare users on db and backup.')

                df_usr_ls.sort()
                db_usr_ls.sort()
                
                if df_usr_ls == db_usr_ls:
                    print('Data job: Users match.')
                else:
                    print('Database job: Users do not match, drop and create users table.')
                    #Create initial users table:
                    print('Database job: Creating users table on DB.')
                    self.query_SQL(queries_path + 'SMI_usrs_table_creation.sql')
                    
                    #Initial users table insertion:
                    print('Database job: Users back into DB.')
                    self.df_to_postgres(conn, df_usr, users_table)
                    print('Database job: Users table inserted on DB.')

In [12]:
#Create class instance:
dbcreate = DatabaseCreation(queries_path, conn, schema)

## Check schema and tables:
dbcreate.db_cs()

## Check backups, tables and fill database:
## Initial users:
dbcreate.insert_ini_users()

## Users:
dbcreate.insert_users()

Database job: Check if SMI schema exists.
Database job: SMI schema exists on DB.
Database job: Check if initial users table exist on DB.
Database job: Initial users table exist on DB.
Database job: Check if users table exist on DB.
Database job: Users table exist on DB.
Data job: Check if initial users backup exists.
Data job: initial users backup exists. Loading file: ../data/db_ini_users_2022-02-08.json
Data job: initial users backup from json file retrieved.
Database job: Initial users table is not empty.
Data job: Compare initial users on DB and backup.
Data job: initial users match.
Data job: Check if users backup exists.
Data job: users backup exists. Loading file: ../data/db_users_2018-06-01.json
Data Engineering job: Filtering location from backup users.
Data Engineering job: Observations before filter: 673327
Data Engineering job: Observations after filter: 240232
Data job: users backup from json file retrieved.
Database job: Users table is empty.
Database job: Insert users ba

In [29]:
path_corpus = '../../../../context/SMI/data/train_model/TASScorpus.json'
with open(path_corpus, 'r') as f:
    df = pd.json_normalize(json.load(f))
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S.%f')
    df['content'] = df['content'].replace('"','', regex=True)
    df['content'] = df['content'].replace(',','', regex=True)
    df['content'] = df['content'].replace(r'\\',' ', regex=True)
    df['sentiment'] = df['sentiment'].replace(',','', regex=True)
    df['content'] = df['content'].replace(r'\r+|\n+|\t+','', regex=True)

In [59]:
df.dtypes

tweetid              object
user                 object
content              object
date         datetime64[ns]
lang                 object
sentiment            object
dtype: object

In [60]:
df[df['tweetid'] == '172131381843984385'].iloc[0, 2]

'Perfectamente imperfecto. Besos RT @CiindyRomero: @AlejandroSanz no lo veo pero lo comienzo a sentir. Será perfecto! 355 240 275 355 262 277'

In [2]:
path_corpus = '../../../../context/SMI/data/municipalities/munlist.json'
with open(path_corpus, 'r') as f:
    df = pd.DataFrame(json.load(f), columns=['location'])
    df.drop_duplicates(inplace = True)

In [3]:
df

,location
0,a arnoia
1,a baña
2,a bola
3,a capela
4,a cañiza
...,...
11696,álava
11697,la tierra llana
11698,cruz de tenerife
11699,de tenerife


In [3]:
def df_to_postgres(df, table, conn):
    """
    Function to save dataframe into postgres with copy_from:
    params:
        - conn: database connection.
        - df: pandas dataframe.
        - table: database table.
    """
    #Buffering the dataframe into memory:
    buffer = StringIO()
    df.to_csv(buffer, header=False, index=False)
    buffer.seek(0)

    #Copy cached dataframe into postgres:
    cur = conn.cursor()
    cur.copy_from(buffer, table, sep=",")
    conn.commit()

def treat_text(df, text_col, date_col = 'date', sent_col = 'sentiment'):
    '''
    Function to treat the corpus columns:
    params:
        - df: dataframe to treat.
    Output: Dataframe treated.
    '''

    #Columns treatment:
    
    df[text_col] = df[text_col].replace(',','', regex=True)
    df[text_col] = df[text_col].replace('"','', regex=True)
    df[text_col] = df[text_col].replace(r'\\',' ', regex=True)
    df[text_col] = df[text_col].replace(r'\r+|\n+|\t+','', regex=True)
    df[text_col]= df[text_col].replace('[^a-zA-Z0-9]', '')
    
    if date_col == 'date':
        df[date_col] = pd.to_datetime(df[date_col])
    if sent_col == 'sentiment':
        df[sent_col] = df[sent_col].replace(',','', regex=True)
    
    return(df)

def insert_tweets(path, dir, file):
    
    try:
        with open(path + dir + '/' + file, 'r') as f:
            df = pd.DataFrame(json.load(f))
        df = treat_text(df, 'text', date_col = None, sent_col = None)
        df_to_postgres(df, 'smi_tweets', conn)
    except Exception as error:
        print(error)

In [26]:
path_tweets = '../../../../context/SMI/data/get_tweets/2015/DF_2015_12_22.json'
with open(path_tweets, 'r') as f:
    df = pd.DataFrame(json.load(f))
df = treat_text(df, 'text', date_col = None, sent_col = None)

In [27]:
df

,username,date,text
0,carlosriverao,2015-12-22 23:42:00,Google is working on an AI-powered chat servic...
1,carlosriverao,2015-12-22 04:20:00,Google and Ford reportedly creating a new comp...
2,RitaMRG,2015-12-23 00:47:00,Por que @LauraMCJ3 es tan amor? Es taaaaaan cu...
3,RitaMRG,2015-12-22 23:09:00,PATÉTICOS! Sofía o aritz? JA! Gh hasta nunkiii...
4,COMBarcelona,2015-12-22 16:48:00,Entre els #ajutsCOMB també trobaràs activitats...
...,...,...,...
267337,Mr_Sutilezas,2015-12-22 11:33:00,@chicadeltirso93 @A3Noticias a que te refieres...
267338,Mr_Sutilezas,2015-12-22 11:10:00,@moedetriana probrabremente
267339,Mr_Sutilezas,2015-12-22 10:23:00,@dramatictone solo falta la publicidad del San...
267340,Mr_Sutilezas,2015-12-22 10:09:00,@perroflautak @dramatictone @_ju1_ las habría ...


In [28]:
if not 'retweets' in (list(df.columns)):
    df['retweets'] = 0
if not 'favorites' in (list(df.columns)):
    df['favorites'] = 0

In [29]:
df[['username', 'date', 'retweets', 'favorites', 'text']]

,username,date,retweets,favorites,text
0,carlosriverao,2015-12-22 23:42:00,0,0,Google is working on an AI-powered chat servic...
1,carlosriverao,2015-12-22 04:20:00,0,0,Google and Ford reportedly creating a new comp...
2,RitaMRG,2015-12-23 00:47:00,0,0,Por que @LauraMCJ3 es tan amor? Es taaaaaan cu...
3,RitaMRG,2015-12-22 23:09:00,0,0,PATÉTICOS! Sofía o aritz? JA! Gh hasta nunkiii...
4,COMBarcelona,2015-12-22 16:48:00,0,0,Entre els #ajutsCOMB també trobaràs activitats...
...,...,...,...,...,...
267337,Mr_Sutilezas,2015-12-22 11:33:00,0,0,@chicadeltirso93 @A3Noticias a que te refieres...
267338,Mr_Sutilezas,2015-12-22 11:10:00,0,0,@moedetriana probrabremente
267339,Mr_Sutilezas,2015-12-22 10:23:00,0,0,@dramatictone solo falta la publicidad del San...
267340,Mr_Sutilezas,2015-12-22 10:09:00,0,0,@perroflautak @dramatictone @_ju1_ las habría ...


In [30]:
df['retweets']

0         0
1         0
2         0
3         0
4         0
         ..
267337    0
267338    0
267339    0
267340    0
267341    0
Name: retweets, Length: 267342, dtype: int64

In [4]:
path_tweets = '../../../../context/SMI/data/get_tweets/'
dirs = os.listdir(path_tweets)

for dir in dirs:
    print("Importing", dir)
    files = os.listdir(path_tweets + dir + '/')
    for file in files:
        print('Importing' , file)
        insert_tweets(path_tweets, dir, file)

Importing 2015
importing DF_2015_11_21.json
importing DF_2015_8_21.json
importing DF_2015_12_21.json
invalid input syntax for type numeric: "Mobile's Workplace Role Continues to Grow http://www. emarketer.com/Article/Mobile s-Workplace-Role-Continues-Grow/1013365 … http://www. emarketer.com/Article/Mobile s-Workplace-Role-Continues-Grow/1013365 …"
CONTEXT:  COPY smi_tweets, line 1, column retweets: "Mobile's Workplace Role Continues to Grow http://www. emarketer.com/Article/Mobile s-Workplace-Role-..."

importing DF_2015_7_21.json
importing DF_2015_10_21.json
importing DF_2015_9_21.json
importing DF_2015_1_21.json


KeyboardInterrupt: 